# BlockMatrix

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.6
SparkUI available at http://hms-beagle-5466c684ff-2l8nm:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.58-3f304aae6ce2
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/hail-20201201-1408-0.2.58-3f304aae6ce2.log


In [116]:
import numpy as np

In [2]:
mt = hl.read_matrix_table('/home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/variants.mt')

In [3]:
mt_small = mt.sample_rows(0.001)

In [4]:
mt_small.count()

(990, 48)

In [18]:
bm = hl.linalg.BlockMatrix.from_entry_expr(mt_small.GT[0], mean_impute=True)

2020-12-01 14:25:02 Hail: INFO: Wrote all 1 blocks of 990 x 48 matrix with block size 4096.


In [19]:
bm.shape

(990, 48)

In [99]:
op = bm[0,:].T @ bm[0,:]

In [100]:
op.shape

(48, 48)

In [101]:
sub = op[0:10,0:10].to_numpy()

In [102]:
sub

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [83]:
bm[0,:].to_numpy()

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

## Pairwise differences

Per sample

In [159]:
diffs = 0
# Only compute the lower triangular.
for ii in range(n_cols):
    for jj in range(ii):
        # Sum of absolute differences.
        diffs += (bm[:, ii] - bm[:, jj]).abs().sum()

# Calculate the average.
avg_pairwise_diff = diffs / (n_cols * (n_cols - 1) / 2)

In [211]:
print('Average pairwise differences between all sequences across all loci: {avg}'.format(avg=avg_pairwise_diff))

Average pairwise differences between all sequences across all loci: 44.84619737437752


Per site.

In [229]:
diff0_1 = (bm[:, 0] - bm[:, 1]).abs()
diff0_2 = (bm[:, 0] - bm[:, 2]).abs()
diff0_1_2 = diff0_1 + diff0_2

In [238]:
bm.element_type

dtype('float64')

In [234]:
bm[:,0].sum()

54.0

In [232]:
diff0_1_2.sum()

100.06382978723404

In [221]:
# The shape should be (number of sequences) x (number of sequences).
n_rows, n_cols = bm.shape

diff_vec = hl.linalg.BlockMatrix.fill(n_rows, 1, 0.0)
n_comp = 0
# Only compute the lower triangular.
for ii in range(n_cols):
    for jj in range(ii):
        n_comp += 1
        # Sum of absolute differences.
        diff_vec += (bm[:, ii] - bm[:, jj]).abs()


In [223]:
#diff_vec[0,0]

In [163]:
# The shape should be (number of sequences) x (number of sequences).
n_rows, n_cols = bm.shape

# Vector of differences at each site.
diff_vec = np.zeros(n_rows)

for ii in range(n_rows):
    diffs = 0
    # Only compute the lower triangular.
    for mm in range(n_cols):
        for nn in range(mm):
            # Sum of absolute differences.
            diffs += abs(bm[ii, mm] - bm[ii, nn])

    # Calculate the average.
    diff_vec[ii] = diffs

KeyboardInterrupt: 

In [ ]:
diff_vec[:10]